## Fetch Tweets of the handles in the dataframes then create csv files for each account.

***WARNING:***The run time is really long, run it only iff you want the tweets of each of the scrapped handles.
May be usefull for hashtags and retweets pulling.

**Assumptions:**
* The code assumes there exists 2 dataframes populated with a column of twitter handles labelled 'handles'.
* The handles are from scrappping the 'top 100' and 'leaders response' site.

In [2]:
import tweepy
import csv
import os
import pandas as pd

consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')

def get_tweets(screen_name):

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []  

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet minus one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab. 
    # Limit set to around 3k tweets, can be edited to preferred number.
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id arg to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200, max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]

    #write the csv  
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)

    pass

In [4]:
# fetch the unique handles from the top_100 and leaders dataframes
# convert to list the merge them to be one list.
the_100 = dataframe_1.handles.unique()
the_leaders_response = dataframe_2.handles.unique()
l1 = the_100.astype(str).tolist() 
l2 = the_leaders_response.astype(str).tolist()
accounts = l1 + l2

if __name__ == '__main__':
    #loop through the handles in the list
    for i,name in enumerate(accounts):
        get_tweets(name) 